<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/kikim6114/NLP2024-1/blob/main/06_summarization-kikim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

이 노트북을 코랩에서 실행하려면 Pro 버전이 필요할 수 있습니다.

In [ ]:
# 코랩을 사용하지 않으면 다음 코드를 주석 처리하세요.
# !git clone https://github.com/kikim6114/nlp-practice-2025.git
# %cd nlp-practice-2025
# from install import *
# install_requirements(chapter=6)

In [ ]:
from transformers import pipeline, set_seed

# 5. 요약

요약(Summarization)이 도전적 작업인 이유:
- 긴 단락의 이해
- 관련 내용의 추론
- 원래 문서의 주제를 통합해 유창한 텍스트를 생성
- 영역별 요약 방법이 다르다(예: 기사와 법률계약서의 요약 방법은 다르다)

## 5.1 CNN/DailyMail 데이터셋

CNN/DailyMail dataset
- 300,000개 뉴스 기사와 요약의 쌍
- 요약을 위해 익명화 처리되지 않은 v3.0.0 사용
- 요약: 본문에서 추출(extractive)하지 않고 추상적(abstractive)이다.
  - Extractive: 본문의 어떤 구간을 가져다 사용함
  - Abstractive: 내용에 맞춰 글을 생성

In [ ]:
!pip install -U datasets

In [ ]:
from datasets import load_dataset, config

# Clear the dataset cache directory to force a fresh download
!rm -rf {config.HF_DATASETS_CACHE}

# "cnn_dailymail" 데이터셋 다운로드 에러가 발생할 경우 대신 "ccdv/cnn_dailymail"을 사용하세요.
# dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0", download_mode="force_redownload")
dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0", trust_remote_code=True)

print(f"특성: {dataset['train'].column_names}")

In [ ]:
sample = dataset["train"][1]
print(f"""기사 (500개 문자 발췌, 총 길이: {len(sample["article"])}):""")
print(sample["article"][:500])
print(f'\n요약 (길이: {len(sample["highlights"])}):')
print(sample["highlights"])

In [ ]:
sample

기사가 긴 경우, 입력 시퀀스 최대 길이가 1024인 Transformer 특성상 뒷부분의 기사에 포함된 정보는 사라진다.

## 6.2 텍스트 요약 파이프라인
- 여러 트랜스포머 기반 요약 모델을 살펴보자.
- 비교를 위해 입력의 길이가 모두 동일하도록 길이를 2000 글자로 제한한다.

In [ ]:
sample_text = dataset["train"][1]["article"][:2000]
# 딕셔너리에 각 모델이 생성한 요약을 저장합니다.
summaries = {}

- 텍스트를 문장 단위로 분리하는 것이 관행이므로,
- 좋은 성능을 갖는 NLTK 패키지를 사용하여 분할한다.

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")
nltk.download('punkt_tab')

In [ ]:
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

### 6.2.1 요약 기준 모델

- 요약된 텍스트의 처음 3 문장만 가져오는 함수를 정의
- 문장 구분은 `\n` 문자

In [ ]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [ ]:
summaries["baseline"] = three_sentence_summary(sample_text)
summaries

### 6.2.2 GPT-2
- 입력 텍스트 끝에 `TL;DR`을 붙여서 요약을 생성하게 할 수 있다.
- `TL;DR`: Too Long; Didn't Read (Reddit 같은 사이트에서 긴 글을 중간에 생략하는 방법으로 사용)
- sample_text + "\nTL;DR:\n" 을 입력하여 🤗Transformers의 `pipeline()`으로 원래 기사를 재생성하는 것으로 실험을 시작한다는 것에 주목하자.

In [ ]:
!pip install hf_xet

In [ ]:
from transformers import pipeline, set_seed

set_seed(42)

# 코랩의 경우 gpt2-xl을 사용하면 메모리 부족 에러가 발생합니다.
# 대신 "gpt" 또는 "gpt2-large"로 지정하거나 코랩 프로를 사용하세요.
pipe = pipeline("text-generation", model="gpt2-large")

gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True) # 불필요한 공백 제거
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))  # pipeout에서 나온 전체 문장 중 모델이 생성한 전체 텍스트 중에서 프롬프트 부분을 제외하고 생성한 부분만 출력

### 6.2.3 T5

<img alt="T5" width="700" caption="Diagram of T5's text-to-text framework (courtesy of Colin Raffel); besides translation and summarization, the CoLA (linguistic acceptability) and STSB (semantic similarity) tasks are shown" src="https://github.com/kikim6114/nlp-practice-2025/blob/main/images/chapter06_T5.png?raw=1" id="T5"/>

- T5: 모든 NLP task들을 text-to-tex 작업으로 구성하는 universal transformer 아키텍처
- T5는 요약을 포함하는 지도 및 비지도 학습데이터로 훈련됨
- Fine-tuning 없이 pretraining에 사용했던 프롬프트를 사용해 바로 요약, 번역

🛠️ 주요 작업 유형(Task Types)
T5는 다음과 같은 NLP 태스크에 모두 활용

✅ 텍스트 요약 (Summarization)

✅ 기계 번역 (Translation)

✅ 문장 분류 (Classification)

✅ 문장 유사도 판단 (Regression)

✅ 질문 생성 및 답변 (QA)

✅ 감정 분석 (Sentiment Analysis)

✅ 개체명 인식 (NER) 등

| 항목                        | 설명                                                                   |
| ------------------------- | -------------------------------------------------------------------- |
| 🧩 **Text-to-Text 통일 구조** | 모든 태스크를 "텍스트 → 텍스트"로 통합하여 일관된 학습 구조 제공                               |
| 🗂 **멀티태스크 학습**           | 다양한 태스크를 하나의 모델에서 동시에 학습 가능                                          |
| 🔧 **프롬프트 기반 제어**         | `"summarize:"`, `"translate:"` 등의 명령어로 태스크 제어 가능                     |
| 🧠 **사전학습 + 파인튜닝**        | 대규모 코퍼스 사전학습 후 다양한 태스크에 맞춰 파인튜닝 가능                                   |
| 📐 **확장성**                | `t5-small` → `t5-base` → `t5-large` → `t5-3b` → `t5-11b` 등 다양한 크기 제공 |


In [ ]:
!pip install huggingface_hub

In [ ]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### 6.2.4 BART

- 트랜스포머를 기반으로한 seq2seq 모델(encoder-decoder)로 구축된 denoising auto encoder
- BERT와 GPT-2 사전훈련 방식을 결합하여 훈련
- 사전학습 중에 BERT 류의 이전 모델들 보다 더 광범위한 노이즈 체계를 제공

<img alt="T5" width="500" src="https://github.com/kikim6114/nlp-practice-2025/blob/main/images/chapter06_bart.png?raw=1" id="bart">

- 여기서는 `CNN/DailyMail`에 미세조정된 `facebook/bart-large-ccn` checkpoint를 사용

Bart는 encoder에서 손상시킨 문장을 복원하는 방법으로 학습되고, decoder에서 기존 transformer에서 학습 하는것과 같이 autoregressive decoding(token을 순서대로 생성)으로 사전학습을 진행하는 모델  

### 📚 BART의 Pretraining 손상 기법 정리
BART는 다양한 손상(Corruption) 기법을 통해 입력 문장을 변형한 후, 원래 문장을 복원하는 방식으로 사전학습을 수행합니다. 이 과정을 통해 문장의 구조와 의미를 깊이 있게 학습할 수 있습니다.

### 🟩 Token Masking
기존의 BERT 모델처럼 무작위의 토큰들을 마스킹합니다.
마스킹된 토큰이 무엇이었는지를 예측하는 방식으로 학습이 진행됩니다.

### 🟥 Token Deletion
무작위로 선택된 토큰들을 삭제합니다.
마스킹과 달리 삭제된 위치에 대한 정보가 주어지지 않아, 해당 위치를 추정해야 하므로 더 어려운 복원 작업이 요구됩니다.

### 🟨 Sentence Permutation
문장 간의 순서를 섞어 노이즈를 추가합니다.
문맥을 무시한 순서 변경을 통해 문장의 전반적 구조 이해 능력을 학습합니다.

### 🟦 Document Rotation
문서 내에서 임의의 토큰을 선택해 해당 위치를 문장의 시작점으로 만들고, 나머지 토큰들을 재배열합니다.
이는 문서의 시작점을 인지하는 능력을 학습하는 데에 활용됩니다.

### 🟪 Text Infilling
논문에서 가장 성능이 좋았다고 언급된 방식입니다.
Poisson 분포(λ=3)에 따라 span 길이를 결정하고, 해당 span을 통째로 마스킹합니다.
이 방식은 문장 내 연속된 공백을 복원하는 능력을 집중적으로 학습시킵니다.
KoBART 등 일부 파생 모델은 이 방식만을 사용해 학습되기도 했습니다.  

![바트의 손상학습 방법](https://github.com/kikim6114/nlp-practice-2025/blob/main/images/BART_encoder_training?raw=1)


In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### 6.2.5 PEGASUS
- Zhang, Jingqing, et al. "Pegasus: Pre-training with extracted gap-sentences for abstractive summarization." International conference on machine learning. PMLR, 2020.

<img alt="pegasus" width="700" caption="Diagram of PEGASUS architecture (courtesy of Jingqing Zhang et al.)" src="https://github.com/kikim6114/nlp-practice-2025/blob/main/images/chapter06_pegasus.png?raw=1" id="pegasus"/>

- 논문 제목에서 알 수 있듯이 Abstractive Summarization에 특화된 모델
- BART와 마찬가지로 Encoder-Decoder 트랜스포머
- Token 단위가 아닌 `Important Sentence` 단위로 masking 하고 이를 생성하는 방법(Gap Sentence Generation; GSG)으로 학습.
- Important sentence: Document 내에서 다른 문장에 비해 전체적인 context를 잘 설명할 수 있는 문장.

| 단계                   | 내용                                         |
| -------------------- | ------------------------------------------ |
| **1. 문장 선택**         | 중요한 문장 2개 선택 (`mythical`, `names`)         |
| **2. 입력 구성**         | 해당 문장을 `[MASK1]`, `[MASK2]`로 치환한 입력 텍스트 생성 |
| **3. 목표 출력(Target)** | 제거된 문장을 순차적으로 복원 (`It is pure white.` 등)   |
| **4. 디코더 입력**        | 출력 문장을 한 칸 오른쪽으로 시프트하여 디코더에 공급             |


In [ ]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

- 이 모델은 자동 줄바꿈하는 특수 토큰이 있어 `sent_tokenize()` 함수가 필요 없음

## 6.3 요약 결과 비교하기

- GPT-2는 데이터셋에서 전혀 훈련되지 않았음을 주목.
- T5는 여러 작업 중의 하나로 이 작업을 위해 미세조정됨.
- BART와 PEGASUS는 이 작업만을 위해 미세조정됨.

In [ ]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

- GPT-2: **Hallucination**, **invented facts**
- 나머지 3개 모델의 요약을 정답과 비교하면 놀랄 정도로 많이 중복되며, 그중에서 PEGASUS가 정답에 가장 가깝다.

실 사용 환경에 사용할 모델의 선택:
- 몇 개의 샘플을 추가로 요약해 보는 것은 최선의 모델을 선택하는은 체계적 방법이 아니다.
- 지표 하나를 정의하고 어떤 벤치마크 데이터셋에서 모든 모델을 평가해서 성능이 최고인 모델을 선택하는 것이 이상적이 방법이다.
- 어떤 지표를 정의해야 텍스트 생성에 우수한 모델을 판별할 수 있을까?
  - accuracy, recall, 그리고 precision 같은 지표는 이 작업에 적용하기 어렵다
  - 사람이 작성한 'Gold standard' 요약마다 동의어, 다른 말로 바꿔 쓰거나, 또는 조금 다르게 작성하는 방식으로 수십개의 요약이 가능하기 때문.

## 6.4 생성된 텍스트 품질 평가하기

- 텍스트 생성 작업의 평가는 표준적인 분류 작업만큼 쉽지 않다.
- 단순히 참조 문장과 후보 문장이 정확히 일치하는지 확인하는 것은 최선이 아니다.
- 사람이 직접 작성한 "Gold Standard" 정답도 여러개일 수 있기 때문이다.

### 6.4.1 BLEU

- BLEU: Bilingual Evaluation Understudy
- Papineni, Kishore, et al. "Bleu: a method for automatic evaluation of machine translation." Proceedings of the 40th annual meeting of the Association for Computational Linguistics. 2002.
- 한 개의 생성된 후보 문장 $sent$ 와 (복수의) 참조 문장 $sent'$ 사이에, 𝑛-gram들이 얼마나 일치하는지를 비교.
- 비교 방법: 유사도(modified 𝑛-gram precision)
- Modified 𝑛-gram Precision:
$$p_n=\frac{\sum_{snt \in C}\sum_{n\mathrm{gram} \in snt'}Count_{\mathrm{clip}}(n\mathrm{gram})}{\sum_{snt' \in C}\sum_{n\mathrm{gram} \in snt}Count(n\mathrm{gram})}$$
- Brevity 페널티 (BR)
$$BR=\mathrm{min}\left(1, e^{1-l_{ref}/l_{gen}}\right)$$
- BLEU Score:  
$$\mathrm{BLEU}_N=BR \times \left(\prod_{n=1}^{N}p_n\right)^{1/N}$$
- BLEU는 동의어를 고려하지 않으며, 유도된 식의 많은 단계에 허술함이 존재한다.
- BLEU의 단점에 대한 참고 문헌:
[Evaluating Text Output in NLP: BLEU at Your Own Risk](https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213)
- 텍스트가 토큰화되어야 하며, 토큰화 방법이 다르면 BLEU score도 달라질 수 있다.
- `SacreBLEU`: 토큰화 단계를 내장시킴 -> 벤치마킹에서 선호되는 방법

In [ ]:
# 영문판 원본의 소스 코드 ==>  실행되나, 나중에 evaluate() 때문에 오류
# from datasets import load_metric

# bleu_metric = load_metric("sacrebleu")

In [ ]:
# 번역판 소스코드
!pip install evaluate
import evaluate
bleu_metric = evaluate.load("sacrebleu")

In [ ]:
import pandas as pd
import numpy as np

bleu_metric.add(
    prediction="the the the the the the", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

- `reference`: list를 전달하는 이유는 참조 문장이 여러개 일 수 있기 때문
- `smooth_method`: zero precision 방지를 위한 smoothing 방법 지정("floor"이면 smooth_value=0.1 이 default)
- `smooth_value`: n-gram이 하나도 없을 때 precision이 0 이되는 것을 방지하기 위함(여기서는 0 으로 할당하여 smoothing 기능을 끔)

In [ ]:
bleu_metric.add(
    prediction="the cat is on mat", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

### 6.4.2 ROUGE
- Lin, Chin-Yew. "Rouge: A package for automatic evaluation of summaries." Text summarization branches out. 2004.
- 번역에서는 가능하고 적절한 모든 단어를 포함하는 번역 보다는 정확도(Precision)가 높은 번역이 선호되므로 기계번역에서는 `BLEU`가 많이 사용된다.
- Summarization 같은 작업에서는 생성된 텍스트에 모든 중요한 정보가 포함되기를 원하므로, 재현율(Recall)이 높기를 원하며, 이런 경우 `ROUGE`가 사용된다.
- BLEU: 후보 문장의 n-gram이 참조 문장에 얼마나 많이 등장하는지 확인
- ROUGE: 참조 문장의 n-gram이 후보 문장에 얼마나 많이 등장하는지 확인 <br>
<br>
<br>
**ROUGE-1** : n-gram recall  

정답 문장: "한화는 10 년 안에 우승 할 것이다."  
생성 문장: "두산은 3 년 안에 우승 할 것이다."  

- $N_{\text{정답문장}} = 7$  
- $N_{\text{일치 단어 수}} = \text{"년", "안", "우승", "할", "것이다"} = 5$  


$$
\text{ROUGE-1} = \frac{N_{\text{일치 단어 수}}}{N_{\text{정답문장}}} = \frac{5}{7}
$$

**ROUCE-2** :

정답 문장: "한화는 10 년 안에 우승 할 것이다."  
생성 문장: "한화는 10 년 안에 절대 우승 못 할 것이다."

- $N_{\text{정답 bigrams}} = 6$  
  - ("한화", "10"), ("10", "년"), ("년", "안"), ("안", "에"), ("우승", "할"), ("할", "것이다")
- $N_{\text{일치 bigrams}} = 4$  
  - ("한화", "10"), ("10", "년"), ("년", "안"), ("할", "것이다")


$$
\text{ROUGE-2} = \frac{N_{\text{일치 bigrams}}}{N_{\text{정답 bigrams}}} = \frac{4}{6}
$$


**ROUGE-L** :  
가장 긴 sequence의 recall을 구함. 이어지지 않아도 됨  

정답 문장: "한화는 10 년 안에 우승 할 것이다."  
생성 문장: "한화는 10 년 안에 절대 우승 못 할 것이다."

- $N_{\text{정답문장}} = 7$
- $\text{Longest Common Subsequence (LCS)} = \text{"한화는 10 년 안에 우승 할 것이다."}$
- $N_{\text{LCS}} = 7$

$$
\text{ROUGE-L} = \frac{N_{\text{LCS}}}{N_{\text{정답문장}}} = \frac{7}{7} = 1
$$


🤗Datasets에서 제공하는 ROUGE 점수 종류:
- ROUGE-L: 문장마다 점수를 계산해서 요약에 대해 평균한 점수
- ROUGE-Lsum: 전체 요약에 대해 직접 계산한 점수

In [ ]:
rouge_metric = evaluate.load("rouge")  # 루지 평가 함수를 담은 객체

In [ ]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

PEGASUS 논문에 따르면, CNN/DaylyMail 데이터셋에서 T5 보다 뛰어나며, 적어도 BART에 견줄만 하다고 주장. PEGASUS 논문의 결과가 재현되는지 살펴보자.

## 6.5 CNN/DailyMail 데이터셋에서 PEGASUS 평가하기

노트북 중간부터 실행하는 경우에만 다음 Cell을 실행할 것!

In [ ]:
# 이 셀은 노트북 중간부터 실행하기 위한 것입니다. 실행하려면 주석처리를 삭제하자
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset
import evaluate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# "cnn_dailymail" 데이터셋 다운로드 에러가 발생할 경우 대신 "ccdv/cnn_dailymail"을 사용하세요.
dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0")
rouge_metric = evaluate.load("rouge", cache_dir=None)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

- 이제 모델을 적절히 평가할 요소를 모두 갖췄다.
- 우선 처음 세 문장을 사용하는 baseline model의 성능부터 평가하자.

In [ ]:
def evaluate_summaries_baseline(dataset, metric,   # 주어진 기사에 대해서 모델이 만든 요약문의 rouge 성능 지표 계산 함수
                                column_text="article",
                                column_summary="highlights"):
    summaries = [three_sentence_summary(text) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries,
                     references=dataset[column_summary])
    score = metric.compute()
    return score

- CNN/DaylyMail 데이터셋의 테스트셋의 샘플수 = ~10,000 개
- 요약 생성에는 많은 시간이 걸린다.
  - 생성되는 모든 토큰이 forward pass를 거쳐야 하므로
  - 샘플마다 100개의 토큰을 생성하려면 100 x 10000 = 1백만번의 forward pass를 거쳐야 함.
- 원래 저자의 책에는 시간을 줄이기 위해 테스트셋에서 1000개을 샘플링하여 사용.
- 실습에서는 시간을 더 줄이기 위해 200개만 사용하기로 함

우선 baseline 모델 평가.

In [ ]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(200))  # 1000을 200으로 수정 - kikim6114

score = evaluate_summaries_baseline(test_sampled, rouge_metric)  # 그냥 기사 원문의 앞에 3문장만 뽑아온 것과의 비교
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

PEGASUS 모델 평가를 위한 함수:

In [ ]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    """list_of_elements로부터 batch_size 크기의 청크를 연속적으로 생성합니다"""
    for i in range(0, len(list_of_elements), batch_size):  # 전체 리스트를 몇개씩 나눠서 진행할 지 결정
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):  # 리스트 두개를 한 번에 쓰기 위해 zip함수 사용

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),  # 요약 생성 과정 수행
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)  # 논문과 동일한 매개변수 값 사용. length_penalty는 요약이 짧게 나오도록 유도

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,  # 위에서 만든 결과 문자로 변환
                                clean_up_tokenization_spaces=True)          # 특수 토큰과 서브워드 구분자 등 불필요한 공백 제거
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)  # references=add_batch는 여러개의 기사를 한 번에 요약할 때 사용

    score = metric.compute()
    return score

이제 seq2seq 생성 작업에 사용되는 `AutoModelForSeq2SeqLM`을 사용하여 모델을 다시 로드하여 평가해본다.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric,
                                   model, tokenizer, batch_size=8)
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

## 6.6 요약 모델 훈련하기
- 앞의 내용을 활용해 Text Summarization 모델을 직접 훈련해보자.
- 삼성이 만든 SAMsum 데이터셋을 사용: 고객센터 대화 내용

In [ ]:
dataset_samsum = load_dataset("knkarthick/samsum")
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"분할 크기: {split_lengths}")
print(f"특성: {dataset_samsum['train'].column_names}")
print("\n대화:")
print(dataset_samsum["test"][0]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][0]["summary"])

### 6.6.1 SAMSum에서 PEGASUS 평가하기
- PUGASUS로 요약 파이프라인을 실행해 어떻게 출력되는지 보자.

In [ ]:
pipe_out = pipe(dataset_samsum["test"][0]["dialogue"])
print("요약:")
print(pipe_out[0]["summary_text"].replace(" .<n>", ".\n"))

SAMsum에서의 요약은 CNN/DailyMail에서보다 추상적이다.

In [ ]:
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model,
                                   tokenizer, column_text="dialogue",
                                   column_summary="summary", batch_size=8)

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

결과가 훌륭하지는 않지만 CNN/DailyMail 데이터셋이 SAMsum 과 크게 다르기 때문에 어느 정도 예상할 수 있는 결과다.
훈련 전에 파이프라인을 준비하면 두 가지 잇점이 있다:
- 훈련이 성공적인지 바로 평가가 가능
- 기준점이 수립됨  

이 데이터셋에서 모델을 미세조정하면 ROUGE 점수가 바로 향상되어야 한다. 그렇지 않으면 훈련과정에 문제가 있는 것이다.

### 6.6.2 PEGASUS 미세 튜닝하기

In [ ]:
d_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["dialogue"]]
s_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["summary"]]

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0")
axes[0].set_title("Dialogue Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")
axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0")
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

- Trainer를 위한 data collator를 만들자
  - 대화 최대 길이 = 1024
  - 요약 최대 길이 = 128

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length=1024,
                                truncation=True)

    with tokenizer.as_target_tokenizer():  # 번역판에서는 context manager with~를 사용하지 않음
        target_encodings = tokenizer(text_target=example_batch["summary"], max_length=128,
                                    truncation=True)

    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,
                                       batched=True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type="torch", columns=columns)

`with tokenizer.as_target_tokenizer()`: 인코더와 디코더의 tokenizer를 구별하는 것이 중요할 때 사용

In [ ]:
# 티처 포싱(teacher forcing)
# 텍스트 생성을 위한 디코더 입력과 레이블의 정렬
text = ['PAD','Transformers', 'are', 'awesome', 'for', 'text', 'summarization']
rows = []
for i in range(len(text)-1):
    rows.append({'step': i+1, 'decoder_input': text[:i+1], 'label': text[i+1]})
pd.DataFrame(rows).set_index('step')

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10, push_to_hub=True,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16)

- `gradient_accumulation_steps=16`: 모델이 크면 batch_size=1 로 하는데, batch가 너무 작으면 수렴하지 않는다.
- 이 문제를 해결하기 위한 기법으로서, 16회 그래디언트가 누적되어 충분히 커지면 최적화 단계가 진행됨
- 학습 속도는 느려지지만 GPU 메모리가 많이 절약됨

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [ ]:
trainer.train()
score = evaluate_summaries_pegasus(
    dataset_samsum["test"], rouge_metric, trainer.model, tokenizer,
    batch_size=2, column_text="dialogue", column_summary="summary")

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"pegasus"])

In [ ]:
trainer.push_to_hub("Training complete!")

### 6.6.3 대화 요약 생성하기

Loss 값과 ROUGE 점수를 보면 CNN/DailyMail에서만 훈련한 원래 모델보다 크게 향상된 것 같다.

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
# `haesun`를 자신의 허브 사용자 이름으로 바꾸세요.
pipe = pipeline("summarization", model="haesun/pegasus-samsum")

print("대화:")
print(sample_text)
print("\n참조 요약:")
print(reference)
print("\n모델 요약:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

- 참조 문장과 훨씬 더 비슷해졌다. 모델이 그냥 문장을 추출하지 않고 대화를 합성해서 요약을 만드는 법을 배운것 같다.
- 실제 대화 입력에서 이 모델은 얼마나 잘 작동할까?

In [ ]:
custom_dialogue = """\
Thom: Hi guys, have you heard of transformers?
Lewis: Yes, I used them recently!
Leandro: Indeed, there is a great library by Hugging Face.
Thom: I know, I helped build it ;)
Lewis: Cool, maybe we should write a book about it. What do you think?
Leandro: Great idea, how hard can it be?!
Thom: I am in!
Lewis: Awesome, let's do it together!
"""
print(pipe(custom_dialogue, **gen_kwargs)[0]["summary_text"])

## 6.7 결론

- Text Summarization은 sentiment analysis, NER, QA 같은 분류 작업들보다 몇가지 특수한 어려움이 있다.
- BLEU나 ROUGE가 있지만 역시 사람의 판단이 가장 좋은 척도다
- Summarization 모델로 작업을 수행할 때, 문맥 크기 보다는 긴 텍스트를 요약하는 방법에 의문을 갖게 된다.
- OpenAI는 긴 문서에 반복적으로 모델을 적용하고 사람의 피드백을 반복 루프에 추가하여 summarization task의 scale을 확장했다.
- Wu, Jeff, et al. "Recursively summarizing books with human feedback." arXiv preprint arXiv:2109.10862 (2021).